In [1]:
import numpy as np
import pandas as pd

import random
import os
from astropy.io import fits
from astropy.table import Table

import matplotlib.pyplot as plt
from astropy.visualization import make_lupton_rgb
plt.style.use('dark_background')

In [2]:
def write_fit_file(name, x, data, catalog):
    if(catalog == 'Jacobs'):
        my_types = {'COADD_OBJECT_ID': int, 'TILENAME': str, 'HPIX_16384': int, 'HPIX_4096': int,
       'DNF_ZMEAN_SOF': float, 'RA': float, 'DEC': float, 'grade': float, 'photoz': float, 'imag': float,
       'FLUX_RADIUS_G': float, 'FLUX_RADIUS_R': float, 'FLUX_RADIUS_I': float, 'FLUX_RADIUS_Z': float,
       'KRON_RADIUS': float, 'GAP_FLUX_G': float, 'MOF_BDF_FLUX_G': float, 'MOF_PSF_FLUX_G': float,
       'SOF_BDF_FLUX_G': float, 'SOF_PSF_FLUX_G': float, 'MAG_AUTO_G': float, 'MAG_APER_4_G': float,
       'MAG_APER_8_G': float, 'SOF_BDF_G_1': float, 'SOF_BDF_G_2': float, 'IMAFLAGS_ISO_G': int,
       'IMAFLAGS_ISO_R': int, 'IMAFLAGS_ISO_I': int, 'IMAFLAGS_ISO_Z': int, 'Candidate': str,
       'objectid': int}
    else:
        my_types = {'COADD_OBJECT_ID': int, 'TILENAME': str, 'HPIX_16384': int, 'HPIX_4096': int,
       'DNF_ZMEAN_SOF': float, 'RA': float, 'DEC': float, 'FLUX_RADIUS_G': float, 'FLUX_RADIUS_R': float,
       'FLUX_RADIUS_I': float, 'FLUX_RADIUS_Z': float, 'KRON_RADIUS': float, 'GAP_FLUX_G': float,
       'MOF_BDF_FLUX_G': float, 'MOF_PSF_FLUX_G': float, 'SOF_BDF_FLUX_G': float, 'SOF_PSF_FLUX_G': float,
       'MAG_AUTO_G': float, 'MAG_APER_4_G': float, 'MAG_APER_8_G': float, 'SOF_BDF_G_1': float,
       'SOF_BDF_G_2': float, 'IMAFLAGS_ISO_G': int, 'IMAFLAGS_ISO_R': int, 'IMAFLAGS_ISO_I': int,
       'IMAFLAGS_ISO_Z': int, 'rank': float, 'num_sources': float, 'num_lenses': float, 'mem_match_id': float,
       'einstein_radius': float, 'einstein_radius_err': float, 'ACT': int, 'BBNA': int, 'BNA': int, 'TI': int,
       'RNA': int, 'RNA2': int, 'redM': int, 'redM_missed': int, 'SPTPol': int}
    data = data.astype(my_types)
    
    primary = fits.PrimaryHDU()
    image = fits.ImageHDU(x, name="IMAGE")
    table_data = Table.from_pandas(data)
    table = fits.BinTableHDU(data = table_data)
    hdu_list = fits.HDUList([primary, image, table])
    hdu_list.writeto('real_lenses/' + name + '.fits', overwrite=True)   
    hdu_list.close()

In [3]:
# Adding some examples of known catalogs: Jacobs
path_cutouts = '/Users/jimenagonzalez/research/DSPL/Searching-double-lenses/real_lenses/Jacobs_Y6/'

hdu_list = fits.open(path_cutouts + 'Cutouts/Jacobs_Y6.fits')
images = hdu_list[1].data.astype(np.float32)
data = hdu_list[2].data
data = Table(data)
data = data.to_pandas()
hdu_list.close()
print('All sample: ', len(images))

data_not = pd.read_csv(path_cutouts + 'not_in_cutouts.csv')
data_not = data_not.astype({'COADD_OBJECT_ID': int})

new_data = pd.merge(data, data_not, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
images = images[new_data.index]
data = data.iloc[new_data.index].reset_index(drop=True)

print('With lensing features inside of cutout: ', len(images))


#Saving a sample for training and other for testing
n_train = 152
idx = random.sample(range(len(data)), n_train)
images_train = images[idx]
data_train = data.iloc[idx]
write_fit_file('Jacobs_train', images_train, data_train, 'Jacobs')

n_test = 228
data_test = data.drop(idx).sample(n_test)
images_test = images[data_test.index]
write_fit_file('Jacobs_test', images_test, data_test, 'Jacobs')

print(len(data_train), len(data_test))

All sample:  496
With lensing features inside of cutout:  457
152 228


In [4]:
# Adding some examples of known catalogs: O'Donnell
path_cutouts = '/Users/jimenagonzalez/research/DSPL/Searching-double-lenses/real_lenses/Jack_Y6/'

hdu_list = fits.open(path_cutouts + 'Cutouts/Jack_Y6.fits')
images = hdu_list[1].data.astype(np.float32)
data = hdu_list[2].data
data = Table(data)
data = data.to_pandas()
hdu_list.close()
print('All sample: ', len(images))


data_not = pd.read_csv(path_cutouts + 'not_in_cutouts.csv')
data_not = data_not.astype({'COADD_OBJECT_ID': int})

new_data = pd.merge(data, data_not, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
images = images[new_data.index]
data = data.iloc[new_data.index].reset_index(drop=True)

print('With lensing features inside of cutout: ', len(images))

#Saving a sample for training and other for testing
n_train = 48
idx = random.sample(range(len(data)), n_train)
images_train = images[idx]
data_train = data.iloc[idx]
write_fit_file('Jack_train', images_train, data_train, 'Jack')

n_test = 72
data_test = data.drop(idx).sample(n_test)
images_test = images[data_test.index]
write_fit_file('Jack_test', images_test, data_test, 'Jack')

print(len(data_train), len(data_test))

All sample:  246
With lensing features inside of cutout:  140
48 72
